In [1]:
import re
def read_sensors(filename = 'input19.txt'):
    with open(filename) as txtfile:
        sensors, idx, positions = {}, 0, []
        for line in txtfile.readlines():
            if '---' in line: idx = int(re.findall('(\d+)', line)[0])
            elif line.strip() == '':
                sensors[idx] = positions
                idx, positions = 0, []
            else: positions.append([int(coordinate) for coordinate in re.findall('(-?\d+)', line)])            
        sensors[idx] = positions
    return sensors

In [2]:
def subtract(position1, position2):
    return [position1[idx] - position2[idx] for idx in range(len(position1))]

In [3]:
def add(position1, position2):
    return [position1[idx] + position2[idx] for idx in range(len(position1))]

In [4]:
def relative_distances(sensor, common_beacons = 12):
    beacons_list = [[sensor[j] for j in range(i, len(sensor))] for i in range(len(sensor)+1-common_beacons)]
    relative_distances = [[subtract(beacon_list[i], beacon_list[0]) for i in range(1, len(beacon_list))] for beacon_list in beacons_list]
    return zip(beacons_list, relative_distances)

In [5]:
def sensors_share_beacons(sensor1, sensor2, common_beacons = 12):
    for beacons_list1, relative_distances1 in relative_distances(sorted(sensor1), common_beacons):
        for beacons_list2, relative_distances2 in relative_distances(sorted(sensor2), common_beacons):
            common_relative_distances = set([tuple(relative_distance) for relative_distance in relative_distances1]).intersection(set([tuple(relative_distance) for relative_distance in relative_distances2]))
            if len(common_relative_distances) >= (common_beacons - 1):
                return True, \
                        [beacons_list1[0]] + [beacons_list1[i+1] for i in range(len(relative_distances1)) if tuple(relative_distances1[i]) in common_relative_distances], \
                        [beacons_list2[0]] + [beacons_list2[i+1] for i in range(len(relative_distances2)) if tuple(relative_distances2[i]) in common_relative_distances]
    return False, None, None

In [6]:
def orientations(sensor):
    def orientations_helper(cube):
        def roll(cube): return [cube[0],cube[2],-cube[1]]
        def turn(cube): return [-cube[1],cube[0],cube[2]]
        orientations = []
        for cycle in range(2):
            for step in range(3):
                cube = roll(cube)
                for i in range(3):
                    orientations.append(cube)
                    cube = turn(cube)
                orientations.append(cube)
            cube = roll(turn(roll(cube)))
        return orientations
    return [[position for position in list_of_positions] \
            for list_of_positions in zip(*[orientations_helper(position) for position in sensor])]

In [7]:
def sensors_positions(sensors):
    sensors_positions = [[0 for _ in range(3)] for _ in range(len(sensors))]
    visited, queue = {0}, [0]
    while (queue):
        new_queue = []
        for i in queue:
            for j in sensors.keys() - visited:
                for sensor_j in orientations(sensors[j]):
                    match, first_list, second_list = sensors_share_beacons(sensors[i], sensor_j, 12)
                    if match:
                        sensors[j] = sensor_j
                        relative_sensor_position_i_to_j = subtract(sorted(first_list)[0], sorted(second_list)[0])
                        sensors_positions[j] = add(sensors_positions[i], relative_sensor_position_i_to_j)
                        new_queue.append(j)
                        visited.add(j)
                        break
        queue = new_queue
    return sensors_positions

In [8]:
def beacons(sensors, sensors_positions):
    beacons = set()
    for sensor in sensors:
        for beacon in sensors[sensor]:
            beacons.add(tuple(add(beacon, sensors_positions[sensor])))
    return beacons

In [9]:
def manhattan_distance(position1, position2):
    return sum(abs(position1[i]-position2[i]) for i in range(len(position1)))    

In [10]:
def max_manhattan_distance(sensors_positions):
    return max(manhattan_distance(sensors_positions1, sensors_positions2) for sensors_positions1 in sensors_positions[:] for sensors_positions2 in sensors_positions[:] )

Part 01

In [11]:
sensors = read_sensors('input19.txt')
sensors_positions = sensors_positions(sensors)
beacons = beacons(sensors, sensors_positions)
answer = len(beacons)
answer

306

Part 02

In [12]:
max_manhattan_distance(sensors_positions)

9764